# Fake News Classifier

Following the <a href="https://data-flair.training/blogs/advanced-python-project-detecting-fake-news/">Data Flair tutorial</a>

In [4]:
import os
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

### Import Data

In [9]:
# Set data path
home = os.getcwd()
news_path = os.path.join(home, "data", "news.csv")

In [11]:
# Read data into a dataframe
df = pd.read_csv(news_path)
print(df.shape)
df.head()

(6335, 4)


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [14]:
# Isolate the labels
labels = df.label

In [21]:
# Split data into test and training sets
x_train, x_test, y_train, y_test = train_test_split(df['text'], labels, test_size = 0.2, random_state = 7)

### TfidVectorizer

In [22]:
# Initialise a vectorizer which discouts stop words and has a 
# maximum document frequency (to be considered words must occur in < 70% of the documents)
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', max_df = 0.7)

# Fit the vectorizer to the training data and use it to transform the test data
tfidf_train = tfidf_vectorizer.fit_transform(x_train) 
tfidf_test = tfidf_vectorizer.transform(x_test)

### PassiveAggressiveClassifier
* In the event of a correct classification, no changes are made (i.e. stayes passive)
* In the event of a misclassification model is adjusted and updated (i.e. becomes aggressive)
* Does not converge (?) 
* Doesn't drastically change the norm of a weight vector (?)

In [23]:
#Initialise a passive aggressive clssifier with maximum iterations of 50 to avoid overfitting
pac = PassiveAggressiveClassifier(max_iter = 50)
pac.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(C=1.0, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=50, n_iter_no_change=5,
                            n_jobs=None, random_state=None, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [25]:
# Predict test data and calculate accuracy of predictons
y_pred = pac.predict(tfidf_test)
score = accuracy_score(y_test, y_pred)
print('Accuracy:', score)

Accuracy: 0.9265982636148382


In [26]:
''' Show number of 
[[True positives, False negatives],
 [False positives, True negatives]]
'''
confusion_matrix(y_test, y_pred, labels=['FAKE', 'REAL'])

array([[588,  50],
       [ 43, 586]], dtype=int64)